## Case Study: Recommendations

<img align="center" style="padding-left:10px; height: 80%; width: 35%" src=http://labs.criteo.com/wp-content/uploads/2017/08/CustomersWhoBought3.jpg>



<a href="https://en.wikipedia.org/wiki/Collaborative_filtering"><img align="right" style="padding-left:10px; height: 40%; width: 40%" src="https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif" ></a>

## General Approach

A generalized version of Collaborative filtering, implied by the adjoining image, is a three-step process:

1. A user expresses their preferences by rating items (e.g. books, movies or CDs) of the system. These ratings can be viewed as an approximate representation of the user's interest in the corresponding domain. _The ratings have been collected by IMDB and imported into the `ratings` DataFrame._
2. The system matches this user's ratings against other users' and finds the people with most "similar" tastes. For the purpose of this Case, we shall determine the recommendations for the user with **userId = 93**.
3. With similar users, the system recommends items that the similar users have rated highly but not yet being rated by this user (presumably the absence of rating is often considered as the unfamiliarity of an item).

### Introduction to the MovieLens datasets.

The accompanying **data** directory contains:

* **`movies_all.csv`**, a list of 9724 movie titles. It's generally referred to as the **the Small Dataset** and comprises of 100,000 ratings and 1,300 tag applications applied to 9,742 movies by 700 users. Its online home is [here](https://grouplens.org/datasets/movielens/latest/).
* **`movies_few.csv`**, a list of 59 movie titles, a subset of `movies_all.csv`,
* **`movies_six.csv`**, a list of 6 movie titles, a subset of `movies_few.csv`,

* **`ratings.csv`**, ratings of all movies in `movies_all.csv`. Corresponding subsets of this file for 'few' or 'six' movies may be obtained by doing a merge, as with

`    .merge(few_movies, how='inner', on='movieId').drop(columns = ['title', 'genres'])` or <br>
`    .merge(six_movies, how='inner', on='movieId').drop(columns = ['title', 'genres'])`

**The Full Dataset:** The Full MovieLens Dataset consisting of 26 million ratings and 750,000 tag applications from 270,000 users on all the 45,000 movies in this dataset can be accessed [here](https://grouplens.org/datasets/movielens/latest/). We will not be using the full dataset in this exercise.

In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
import random
ratings = pd.read_csv('data/ratings.csv') 
movie_ids = ratings[['movieId']]
movies = pd.read_csv('data/movies-all.csv') 
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


<span style="color:blue">

### 1. Raters per Movie
</span>

How many user ratings per movie are in the **`all`** dataset on average? 

In [2]:
len(ratings) / len(set(ratings['userId'].tolist()))

165.30491803278687

<span style="color:blue">

## Solution Development
</span>

We proceed with the calculations as outlined above but first use the **six dataset** such that we can develop the solution and _verify the calculations manually._ 

Step 2 of the algorithms is for the system to match this user's ratings against other users' and finds the people with most "similar" tastes. We shall use **userId = 93** for this tiny dataset.

To formulate the math behind the distance calculation, consider two users $U_{i}$ and $U_{j}$. The “distance” between them, $\Delta_{ij}$, is expressed as

$\Delta_{ij}$ = $\sqrt{ \Sigma {(r_{ik} - r_{jk} )^2 } }$ for all movies <em>k</em> that have been rated by both $U_{i}$ _and_ $U_{j}$, where $ r_{ik} $ and $ r_{jk} $ are ratings of movie _k_ by $U_{i}$ and $U_{j}$

In [3]:
# This cell intentionally left filled

# Initial Parameters
given_userId = 93
movies_six = pd.read_csv('data/movies-six.csv',header = 0) 
movies_six

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Avatar (2009),Fantasy|Science_Fiction
5,6,Titanic (1997),Drama|Disaster_Film


In [4]:
# Make sure the given userId has rated at least two movies, otherwise we wouldn't be able to recommend anything!

ratings_six = ratings.merge(movies_six, how='inner', on='movieId').drop(columns = ['title', 'genres'])
userIds = [el[0] for el in list(np.array(ratings_six[['userId']]))]
seen = {}
dupes = []

for x in userIds:
    if x not in seen:
        seen[x] = 1
    else:
        if seen[x] == 1:
            dupes.append(x)
        seen[x] += 1
try:
    assert(given_userId in dupes)
except AssertionError:
    print ('Should have chosen a userId from amongst these\n', sorted(dupes))
    raise

### Variable Naming Conventions

We will be using numpy and scipy for most of the calculations, mostly use `pandas` for pretty printing. 

Variable names will be chosen such that:

1. Variables ending in `_np` will be used for numpy arrays.
1. Variables ending in `_2d` will be used for 2-D numpy arrays.
1. Variables ending in `_1d` will be used for 2-D numpy arrays.
1. Variables ending in `_df` will be used for pandas DataFrames.

In [5]:
# This cell intentionally left filled

# Find all the ratings given by our user
ratings_six_np = ratings_six.to_numpy(dtype=np.float32)
x = ratings_six_np

# (Movie, Rating) tuples for the given user
the_r_2d = x[np.where(x[:,0] == given_userId)][:, [1,2]]
print (the_r_2d)
the_r_1d = x[np.where(x[:,0] == given_userId)][:, [2]]
the_r_1d

[[1. 3.]
 [2. 5.]]


array([[3.],
       [5.]], dtype=float32)

<span style="color:blue">

### 2. Calculation of Distances
</span>

The Distance matrix looks like
$$
\begin{equation*}
\mathbf{\Delta} =  \begin{vmatrix}
\mathbf{\delta_{11}} & {\delta_{12}} & ..  & {\delta_{1n}} \\
\mathbf{\delta_{21}} & {\delta_{22}} & ..  & {\delta_{2n}} \\
\mathbf{:}           & {:} & ..  & {:} \\
\mathbf{\delta_{n1}} & {\delta_{n2}} & ..  & {\delta_{mn}}
\end{vmatrix}
\end{equation*}
$$

where $ \delta_{ij} $ is the euclidian distance between the $ i^{th} $ and the $ j^{th} $ elements. $ \delta_{ij} $ values are calculated using $ scipy.spatial.distance.euclidean() $ but we needn't calculate them individually. $scipy.spatial.distance.cdist()$ calculates the entire matrix for us with one call!

In [ ]:
# This cell intentionally left filled

# Find the ratings by all users

# The 'u' column -- user
all_u_1d = ratings_six_np[:, [0]]
all_u = all_u_1d.reshape(all_u_1d.shape[0])

# The 'm' column -- movies
all_m_1d = ratings_six_np[:, [1]]
all_m = all_m_1d.reshape(all_m_1d.shape[0])

# The 'r' column -- rating
all_r_1d = ratings_six_np[:, [2]]
all_r = all_r_1d.reshape(all_r_1d.shape[0])

(all_u.shape, all_m.shape, all_r.shape)

In [7]:
from scipy.spatial import distance_matrix
dm = distance_matrix(all_r_1d, all_r_1d)
dm.shape

(535, 535)

<span style="color:blue">

### 3. Verification of Calculations
</span>

For an intermediate step, we would like to validate the distance calculation. It's not because we don't trust Numpy, Scipy, etc. it's because we want to know that we are using them correctly.

In [8]:
# This cell intentionally left filled

# For the manual verification, we randomly pick one of the rows of the_r_1d
from scipy.spatial.distance import cdist

random_index = random.randint(0,len(the_r_1d)-1)
one_r_1d = the_r_1d[random_index].reshape(1,1)

dists = cdist(one_r_1d, all_r_1d)

In [9]:
# This cell is supposed to throw an exception if the results of euclidean calculation don't match
from scipy.spatial.distance import euclidean
from math import sqrt
distances = list(dists.reshape(dists.shape[1]))
one_r = one_r_1d.reshape(the_r_1d.shape[1])
assert(np.isclose(euclidean(one_r, all_r), # Fill in
                  sqrt(sum([_*_ for _ in distances]))))

<span style="color:blue">

### 4. Generate Recommendations
</span>

Recommend items that have been rated similarly (and highly) by similar users but if a specific item has not yet been rated by this user, it is a candidate for being recommended to them. Order the recommendations in _increasing order of distance_.

Your answer should be in the form of a DataFrame `recommend_df` which has rows of ('movieId', 'title', 'genres') ordered with ascending 'r' values, and not including the movies the user 'u' has already watched.

In [10]:
dists = np.stack([all_u,
                  all_m,
                  all_r])
dists_df = DataFrame(dists.transpose(), columns=['u', 'm', 'r']) \
           .astype({'u': 'int32', 'm': 'int32', 'r': 'float32'})

# What movies has the user picked already? Don't recommend those!
candidates_df = dists_df[dists_df['u'] == given_userId].sort_values(by=['r'])
recommend_df = candidates_df.merge(movies_six, left_on='m', right_on='movieId').drop(columns=['u', 'm', 'r'])
recommend_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


<span style="color:blue">

### 5. More To Do
</span>

1. Create a function `recommend_movies(uid)` that takes `userId`, uses one of the `movies_*` as parameters and produces recommendations for the user. 

    * Any userId in \[ 1, 6, 18, 19, 21, 27, 31, 32, 43, 44, 45, 51, 57, 58, 62, 64, 66, 68, 82, 84, 91, 93, 102, 103, 107, 112, 117, 121, 135, 140, 144, 150, 151, 153, 160, 166, 169, 177, 179, 181, 182, 186, 191, 200, 202, 217, 219, 220, 226, 229, 232, 239, 240, 249, 266, 269, 270, 274, 276, 282, 288, 294, 298, 304, 305, 307, 308, 314, 318, 321, 322, 323, 330, 336, 337, 339, 347, 353, 357, 359, 368, 372, 373, 380, 381, 385, 389, 391, 411, 414, 425, 432, 434, 436, 437, 438, 448, 451, 456, 458, 469, 470, 474, 476, 477, 480, 483, 484, 489, 490, 492, 501, 509, 517, 521, 524, 525, 534, 544, 555, 559, 561, 570, 573, 580, 588, 590, 592, 594, 597, 599, 600, 602, 603, 604, 605, 608, 610 \] should work as a test case.


2. Test the function `recommend_movies()` with userId = 117 & 517 against `movies_six`, `movies_few` and against `movies_all`. Document your observations.



# When you're done, submit the notebook

1. **Run all the cells in order.**

2. Submit the notebook by saving it as PDF. 
    * In the cluster environment, it's File | Print (Save as PDF) and submit to [Gradescope](https://www.gradescope.com/courses/182658)<sup>&dagger;</sup>, 
    * On other versions, it may be File | Download As (PDF) and then submit to [Gradescope](https://www.gradescope.com/courses/182658)<sup>&dagger;</sup>.

<sup>&dagger;</sup>To submit to Gradescope, log into the website, add course 9W7PW3 (if not already added) and submit. The assignment name should match the name of this notebook.

![The end](https://live.staticflickr.com/32/89187454_3ae6aded89_b.jpg)